In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d394cc1c203d27d1d93022bc5e7446fc3897346a2f9ac3d03665ba2029aa7425
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer, util

In [3]:
sentences = ["I'm happy", "I'm full of happiness"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
embedding_1= model.encode(sentences[0], convert_to_tensor=True)
embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

util.pytorch_cos_sim(embedding_1, embedding_2)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

tensor([[0.6003]], device='cuda:0')

In [4]:
import pandas as pd
import re

In [5]:
def DescriptionCleaning(df, columns, name):
  columns_list = []
  columns_list.append(columns)
  columns_list.append(name)
  print(columns_list)
  df = df[columns_list]
  df=df.dropna()

  labelRaw = list(df[columns])
  label = []
  for text in labelRaw:
    text = str(text)
    temp = re.sub('[^a-zA-Z]+', ' ', text)
    temp = temp.lower().strip()
    label.append(temp)

  df[columns]=label

  return df

In [6]:
file_path = "/content/Accidents_data_Processed.csv"
df = pd.read_csv(file_path)
df = DescriptionCleaning(df, "Description", "BUSINESS NAME")

['Description', 'BUSINESS NAME']


In [7]:
df.head()

,Description,BUSINESS NAME
0,on february th at am pst a waymo autonomous ve...,Waymo LLC
1,on february th at am pm pst a waymo autonomous...,Waymo LLC
2,on february at am pst a waymo autonomous vehic...,Waymo LLC
3,on february st a test vehicle operating in man...,Apple Inc.
4,a zoox vehicle in manual mode was proceeding s...,Zoox


In [9]:
file_path = "/content/Image to text output.xlsx"
manual_driving_df = pd.read_excel(file_path)
df1 = DescriptionCleaning(manual_driving_df, "Llava output", "Video name")

['Llava output', 'Video name']


In [10]:
df1.head()

,Llava output,Video name
0,a car was driving down a busy street at night ...,CLIP_5.mp4
1,a car was driving down a highway with three ot...,CLIP_9.mp4
2,a car was driving down a street when it sudden...,CLIP_19.mp4
3,a white car was driving down the street when i...,CLIP_20.mp4
4,a car accident occurred on a busy street at ni...,CLIP_26.mp4


In [11]:
count = 0
data = []
obj = {}
# Creating Empty DataFrame and Storing it in variable df
outputDF = pd.DataFrame()
for i in range(len(df)):
    for j in range(len(df1)):
        obj = {"business_name": "", "prompt1": "", "video_name": "", "prompt2": "", "score": 0}
        row1 = df.iloc[i]
        row2 = df1.iloc[j]
        if i != j:
            prompt1 = row1['Description']
            prompt2 = row2['Llava output']
            embedding_1= model.encode(prompt1, convert_to_tensor=True)
            embedding_2 = model.encode(prompt2, convert_to_tensor=True)
            cosine_score = util.pytorch_cos_sim(embedding_1, embedding_2)
            obj["business_name"] = row1["BUSINESS NAME"]
            obj["prompt1"] = prompt1
            obj["video_name"] = row2["Video name"]
            obj["prompt2"] = prompt2
            obj["score"] = cosine_score
            data.append(obj)
            if cosine_score > 0.70:
              count = count + 1
              #print(f"Similarity between Prompt {i+1} and Prompt {j+1}: {cosine_score.item()}")
              #print(prompt1)
              #print(prompt2)
              print(i+1)

print(count)

13
1


In [12]:
outputDF = pd.DataFrame(data)

In [13]:
outputDF.to_excel('Sentence_transformer_similarity_score.xlsx', sheet_name = 'LLava_comparision')

In [14]:
df1 = DescriptionCleaning(manual_driving_df, "VQA and LLaMA Output", "Video name")
count = 0
data = []
obj = {}
# Creating Empty DataFrame and Storing it in variable df
outputDF = pd.DataFrame()
for i in range(len(df)):
    for j in range(len(df1)):
        obj = {"business_name": "", "prompt1": "", "video_name": "", "prompt2": "", "score": 0}
        row1 = df.iloc[i]
        row2 = df1.iloc[j]
        if i != j:
            prompt1 = row1['Description']
            prompt2 = row2['VQA and LLaMA Output']
            embedding_1= model.encode(prompt1, convert_to_tensor=True)
            embedding_2 = model.encode(prompt2, convert_to_tensor=True)
            cosine_score = util.pytorch_cos_sim(embedding_1, embedding_2)
            obj["business_name"] = row1["BUSINESS NAME"]
            obj["prompt1"] = prompt1
            obj["video_name"] = row2["Video name"]
            obj["prompt2"] = prompt2
            obj["score"] = cosine_score
            data.append(obj)
            if cosine_score > 0.70:
              count = count + 1
              #print(f"Similarity between Prompt {i+1} and Prompt {j+1}: {cosine_score.item()}")
              #print(prompt1)
              #print(prompt2)
              print(i+1)

print(count)

['VQA and LLaMA Output', 'Video name']
19
34
40
47
65
68
72
92
106
116
125
158
163
185
189
208
213
215
226
232
233
234
242
243
246
247
250
257
263
273
279
322
329
331
338
360
361
370
372
374
383
387
395
403
408
45


In [15]:
outputDF = pd.DataFrame(data)

In [16]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

path = r"/content/Sentence_transformer_similarity_score.xlsx"

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

outputDF.to_excel(writer, sheet_name = 'VQA_and_LLama')
writer.close()

<ipython-input-16-eeb70b3e7645>:9: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book


In [18]:
df1 = DescriptionCleaning(manual_driving_df, "BLIPVQA 13B and LLaMA Output", "Video name")
count = 0
data = []
obj = {}
# Creating Empty DataFrame and Storing it in variable df
outputDF = pd.DataFrame()
for i in range(len(df)):
    for j in range(len(df1)):
        obj = {"business_name": "", "prompt1": "", "video_name": "", "prompt2": "", "score": 0}
        row1 = df.iloc[i]
        row2 = df1.iloc[j]
        if i != j:
            prompt1 = row1['Description']
            prompt2 = row2['BLIPVQA 13B and LLaMA Output']
            embedding_1= model.encode(prompt1, convert_to_tensor=True)
            embedding_2 = model.encode(prompt2, convert_to_tensor=True)
            cosine_score = util.pytorch_cos_sim(embedding_1, embedding_2)
            obj["business_name"] = row1["BUSINESS NAME"]
            obj["prompt1"] = prompt1
            obj["video_name"] = row2["Video name"]
            obj["prompt2"] = prompt2
            obj["score"] = cosine_score
            data.append(obj)
            if cosine_score > 0.70:
              count = count + 1
              #print(f"Similarity between Prompt {i+1} and Prompt {j+1}: {cosine_score.item()}")
              #print(prompt1)
              #print(prompt2)
              print(i+1)

print(count)

['BLIPVQA 13B and LLaMA Output', 'Video name']
19
38
38
55
72
72
75
91
109
113
125
169
185
189
208
226
232
242
243
246
253
273
280
301
322
326
331
331
331
361
370
372
373
374
380
381
383
387
391
395
397
403
408
411
44


In [19]:
outputDF = pd.DataFrame(data)

In [20]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

path = r"/content/Sentence_transformer_similarity_score.xlsx"

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

outputDF.to_excel(writer, sheet_name = 'BLIPVQA_13B_and_LLaMA')
writer.close()

<ipython-input-20-93ffb6118295>:9: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
